In [24]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras.models import Sequential, load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.regularizers import l2

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import pickle
import os

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

In [15]:
def calculate_and_log_training_time(modelname, start_time, end_time):
    training_duration = end_time - start_time
    hours, rem = divmod(training_duration, 3600)
    minutes, seconds = divmod(rem, 60)
    
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    filepath = os.path.join(model_dir, "trainingtime.txt")
    with open(filepath, "w") as f:
        f.write(f"Training took {int(hours):02d}:{int(minutes):02d}:{seconds:02f} (hh:mm:ss).")

def save_model_config_with_optimizer(model, modelname):
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    filepath = os.path.join(model_dir, "model_config.txt")
    with open(filepath, 'w') as f:
        for layer in model.layers:
            f.write(f"Layer: {layer.name}\n")
            f.write(f"Config: {layer.get_config()}\n\n")
        
        optimizer_config = model.optimizer.get_config()
        f.write("Optimizer Config:\n")
        f.write(str(optimizer_config))

def save_model(model, modelname):
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    model_path = os.path.join(model_dir, modelname + ".keras")
    model.save(model_path)

def save_performance_metrics(history, modelname):
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    filepath = os.path.join(model_dir, 'performance_metrics.csv')
    pd.DataFrame(history.history).to_csv(filepath)

def plot_loss_and_metric(history, metric_name='accuracy', model_name='model'):
    model_dir = model_name
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    # First plot: Training & validation loss
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(history.history['loss'], label='Train Loss')
    ax.plot(history.history['val_loss'], label='Validation Loss')
    ax.set_title('Model Loss')
    ax.set_ylabel('Loss')
    ax.set_xlabel('Epoch')
    ax.legend(loc='upper right')
    ax.grid(True)
    loss_plot_filename = os.path.join(model_dir, f'{model_name}_loss_plot.jpg')
    fig.savefig(loss_plot_filename)
    print(f"Loss plot saved as: {loss_plot_filename}")
    plt.close(fig)
    
    # Second plot: Training & validation metric
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(history.history[metric_name], label=f'Train {metric_name.capitalize()}')
    ax.plot(history.history[f'val_{metric_name}'], label=f'Validation {metric_name.capitalize()}')
    ax.set_title(f'Model {metric_name.capitalize()}')
    ax.set_ylabel(metric_name.capitalize())
    ax.set_xlabel('Epoch')
    ax.legend(loc='upper left')
    ax.grid(True)
    metric_plot_filename = os.path.join(model_dir, f'{model_name}_{metric_name}_plot.jpg')
    fig.savefig(metric_plot_filename)
    print(f"Metric plot saved as: {metric_plot_filename}")
    plt.close(fig)

def save_model_and_config_and_metrics(model, history, modelname = "model"):
    save_model_config_with_optimizer(model, modelname = modelname)
    save_model(model, modelname = modelname)
    save_performance_metrics(history, modelname = modelname)
    plot_loss_and_metric(history, metric_name='mae', model_name= modelname)

## Transfer Learning RESNET50

In [16]:
path = r"C:\Users\busjo\Documents\JADS\Semester 2\Deep Learning\Project\Part1_Processed_RGB.pkl"
# Open the pickle file in binary mode
with open(path, 'rb') as file:
    # Load the content of the file into a variable
    RGB_data = pickle.load(file)

In [17]:
sampled_RGB_data = RGB_data.sample(n=1000, random_state=2001)
y = sampled_RGB_data['Age'].values
X = sampled_RGB_data['Image'].values
X = np.stack(X)  # This should result in a numpy array of shape (num_samples, 256, 256, 3)

# Function to resize and normalize images
def preprocess_images(images):
    """Resize images to 224x224 and normalize them."""
    images = tf.image.resize(images, [224, 224], method='bilinear')
    # Normalize images to [-1, 1] if using a model pretrained on ImageNet
    images = (images / 127.5) - 1
    return images.numpy()  # Convert back to numpy array

# Apply preprocessing to all images
X = preprocess_images(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
base_model = ResNet50(weights='imagenet', include_top=False)
base_model.trainable = False  # Freeze the convolutional base

# Define the model using the Sequential API
transfer_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),  # Additional dense layer with relu activation
    Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model
transfer_model.compile(optimizer='adam', loss='mse', metrics=['mae'])


start_time = time.time()
history = transfer_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
end_time = time.time()

modelname = 'transfer_model_ResNet50'
calculate_and_log_training_time(modelname = modelname, start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = transfer_model, history = history,  modelname = modelname)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 1163.8221 - mae: 26.5446 - val_loss: 559.7067 - val_mae: 19.9884
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 679.8377 - mae: 22.5845 - val_loss: 620.6669 - val_mae: 22.0794
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 650.6486 - mae: 21.9804 - val_loss: 578.5123 - val_mae: 20.9579
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - loss: 630.3478 - mae: 21.2781 - val_loss: 587.5080 - val_mae: 21.2427
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - loss: 633.4879 - mae: 21.7047 - val_loss: 588.7366 - val_mae: 21.2793
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - loss: 614.9753 - mae: 21.2268 - val_loss: 591.9930 - val_mae: 21.3728
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 630.8163 - mae: 21.4023 - val_loss: 586.5230 - val_mae: 21.2131
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - loss: 629.1222 - mae: 21.2988 - val_loss: 584.7124 - val_mae: 21.1589
Epoch 9/20
20/20 ━━━━━━